# Baseline: Logistic Regression and Naive Bayes SVM

To start out the modelling process, we will create two linear classifiers with a bag of words approach. Note here that linear models are generally used with bag of words because the feature vectors are so sparse. We start with a linear regression paired with TFIDF Vectorization.

### TFIDF Bag-of-Words + Logistic Regression

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegressionCV



In [5]:
docs = pd.read_csv('C:\\Users\\harri\\.kaggle\\competitions\\jigsaw-toxic-comment-classification-challenge\\train.csv\\train.csv')
X = docs.iloc[:,1]
y = docs.iloc[:,2:]
class_names = list(docs.columns)[2:]


['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

In [7]:
tfidf = TfidfVectorizer(strip_accents = 'unicode'
                        ,
                        analyzer = 'word',
                        stop_words = 'english',
                        max_features = 10000 )
X = tfidf.fit_transform(X)


In [10]:
CV_scores = []
for name in class_names:
    y = docs[name]
    clf = LogisticRegressionCV()
    score = np.mean(cross_val_score(clf, X, y, cv = 5, scoring = 'roc_auc'), n_jobs = -1)
    print(name, score)
    CV_scores.append(score)

toxic 0.9665574807685318
severe_toxic 0.9820969360575456
obscene 0.9769906339535066
threat 0.9759194963626536
insult 0.9724214566072116
identity_hate 0.971437504256782


In [11]:
np.mean(CV_scores)

0.9742372513343719

In [3]:
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from scipy.sparse import hstack

class_names = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

train = pd.read_csv('C:\\Users\\harri\\.kaggle\\competitions\\jigsaw-toxic-comment-classification-challenge\\train.csv\\train.csv').fillna(' ')
test = pd.read_csv('C:\\Users\\harri\\.kaggle\\competitions\\jigsaw-toxic-comment-classification-challenge\\test.csv\\test.csv').fillna(' ')

train_text = train['comment_text']
test_text = test['comment_text']
all_text = pd.concat([train_text, test_text])

word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    token_pattern=r'\w{1,}',
    stop_words='english',
    ngram_range=(1, 1),
    max_features=10000)
word_vectorizer.fit(all_text)
train_word_features = word_vectorizer.transform(train_text)
test_word_features = word_vectorizer.transform(test_text)

char_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='char',
    stop_words='english',
    ngram_range=(2, 6),
    max_features=50000)
char_vectorizer.fit(all_text)
train_char_features = char_vectorizer.transform(train_text)
test_char_features = char_vectorizer.transform(test_text)

train_features = hstack([train_char_features, train_word_features])
test_features = hstack([test_char_features, test_word_features])

scores = []
submission = pd.DataFrame.from_dict({'id': test['id']})
for class_name in class_names:
    train_target = train[class_name]
    classifier = LogisticRegression(C=0.1, solver='sag')

    cv_score = np.mean(cross_val_score(classifier, train_features, train_target, cv=3, scoring='roc_auc'))
    scores.append(cv_score)
    print('CV score for class {} is {}'.format(class_name, cv_score))

    classifier.fit(train_features, train_target)
    submission[class_name] = classifier.predict_proba(test_features)[:, 1]

print('Total CV score is {}'.format(np.mean(scores)))

CV score for class toxic is 0.9692158485041471
CV score for class severe_toxic is 0.9875939423918284
CV score for class obscene is 0.9838697116279985
CV score for class threat is 0.9833765397623327
CV score for class insult is 0.977426384959597
CV score for class identity_hate is 0.9739429127138002
Total CV score is 0.9792375566599508


This is OK for a baseline, we scored on  the 30th percentile based on our cross validation. However if we want to score in the top 25% we need a ROC-AUC score of  0.9858 . Next lets try a Support Vector Machine paired with Naive Bayes scaling of our features.

### NB-SVM


In [ ]:
import